#  Gradio

https://gradio.app/

Gradio is the fastest way to demo your machine learning model with a friendly web interface so that anyone can use it, anywhere!


In [4]:
!pip install gradio==3.14.0
!pip install --upgrade httpx
!pip install --upgrade httpcore
!pip install tensorflow==2.12.0
!pip install keras==2.12.0
!markupsafe==2.0.1

In [5]:
import numpy as np
from keras.models import load_model

model = load_model('model.h5')

def classify_image(image):
    
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)

    # Obtiene las etiquetas de las clases
    class_labels = ['HOMBRE', 'MUJER']

    # Devuelve un diccionario con las etiquetas de las clases y sus probabilidades correspondientes
    return {class_labels[i]: float(prediction[0][i]) for i in range(2)}


2023-11-19 16:16:57.868543: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.
2023-11-19 16:16:57.910380: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.
2023-11-19 16:16:57.930723: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.
2023-11-19 16:16:58.026229: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.
2023-11-19 16:16:58.075023: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 34560000 exceeds 10% of free system memory.


In [6]:
model = load_model('model.h5')

In [5]:
import gradio as gr

iface = gr.Interface(
    fn=classify_image,
    inputs=gr.components.Image(shape=(150, 150)),  # Actualizado para Gradio >= 4.0
    outputs=gr.components.Label(num_top_classes=2)
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://4ac9dc5e-1df0-426e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 456ms/step


In [ ]:
#pip uninstall markupsafe

# Quantization

La cuantificación se refiere a reducir la precisión numérica de los pesos y las activaciones de un modelo, lo que permite reducir el tamaño y mejorar la eficiencia del modelo. 

TensorFlow proporciona herramientas y APIs para realizar la cuantificación de manera sencilla.

In [1]:
!pip install tensorflow_model_optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.0.0
    Uninstalling absl-py-2.0.0:
      Successfully uninstalled absl-py-2.0.0


In [7]:
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmptoitnt_k/assets


INFO:tensorflow:Assets written to: /tmp/tmptoitnt_k/assets
2023-11-19 16:17:14.879126: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-19 16:17:14.879180: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-19 16:17:14.888531: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmptoitnt_k
2023-11-19 16:17:14.889170: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-11-19 16:17:14.889189: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmptoitnt_k
2023-11-19 16:17:14.901689: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-11-19 16:17:14.902286: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-19 16:17:15.062286: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

#### Probamos: quantization

In [8]:
def classify_image_2(image):
    
    image = np.expand_dims(image, axis=0)

    interpreter = tf.lite.Interpreter(model_path="quantized_model.tflite")
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], image)

    # Ejecuta la inferencia
    interpreter.invoke()

    # Obtiene el resultado de la inferencia
    prediction = interpreter.get_tensor(output_details[0]['index'])

    # Obtiene las etiquetas de las clases
    class_labels = ['HOMBRE','MUJER']

    # Devuelve un diccionario con las etiquetas de las clases y sus probabilidades correspondientes
    return {class_labels[i]: float(prediction[0][i]) for i in range(2)}

In [9]:
import gradio as gr

iface = gr.Interface(
    fn=classify_image_2,  # la función que hace la clasificación
    inputs=gr.inputs.Image(shape=(150, 150)),  # el tipo de entrada que espera tu modelo
    outputs=gr.outputs.Label(num_top_classes=2),  # el tipo de salida que produce tu modelo
)
iface.launch(share=True)

/home/codespace/.python/current/lib/python3.10/site-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/codespace/.python/current/lib/python3.10/site-packages/gradio/outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://decc1eeb-9195-406f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/site-packages/gradio/routes.py", line 321, in run_predict
    output = await app.blocks.process_api(
  File "/home/codespace/.python/current/lib/python3.10/site-packages/gradio/blocks.py", line 1006, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "/home/codespace/.python/current/lib/python3.10/site-packages/gradio/blocks.py", line 847, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/codespace/.python/current/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/codespace/.python/current/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/home/codespace/.python/current/lib/python3.10/site-packages

In [13]:
#!pip show gradio

# Pruning

El pruning, o poda, es una técnica utilizada para reducir el tamaño de un modelo eliminando los pesos que son pequeños o cero.

El concepto es similar a la poda en la jardinería, donde se eliminan las ramas innecesarias para mantener el árbol saludable.

Esto puede mejorar la eficiencia del modelo en términos de velocidad y tamaño de almacenamiento, a veces con un coste mínimo en términos de precisión.

In [ ]:
#!pip install tensorflow-model-optimization

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from tensorflow_model_optimization.sparsity import keras as sparsity

# Cargar y normalizar el conjunto de datos 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convertir las etiquetas en one-hot
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define el preprocesamiento de la imagen

# Define tu modelo
model = tf.keras.models.Sequential([
 
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compila y entrena el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#history = model.fit(x_train, y_train, batch_size=256, epochs=1, validation_data=(x_test, y_test))

# Definir el esquema de pruning. En este caso, comenzamos sin pruning y terminamos con el 50% de los pesos pruned. 
# El pruning comienza en el paso 2000 y termina en el paso 4000.
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                   final_sparsity=0.5,
                                                   begin_step=2000,
                                                   end_step=4000)
}

# Envolver el modelo con las capas de pruning. Esto agregará una operación de pruning a cada capa de nuestro modelo.
model_for_pruning = sparsity.prune_low_magnitude(model, **pruning_params)

# Necesitamos recompilar el modelo después de agregar las capas de pruning.
model_for_pruning.compile(optimizer='adam', 
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

# Entrenar el modelo con callbacks para habilitar pruning. 
# El callback UpdatePruningStep asegurará que el estado del pruning se actualiza en cada paso de entrenamiento.
callbacks = [
  sparsity.UpdatePruningStep(),
]

# Ajustar el modelo con nuestros datos de entrenamiento. Durante este entrenamiento, los pesos del modelo serán pruned según el esquema que definimos antes.
model_for_pruning.fit(x_train, y_train,
                      epochs=1,
                      callbacks=callbacks)

# Guarda el modelo#
model_for_pruning.save('pruning_conv.h5')